In [1]:
import pandas as pd

In [67]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv("../data/uncleaned/brainwaves_data_60100381.csv")

In [5]:
df.head()

,type,timestamp,laptop_timestamp,participantName,raw_CP3,raw_C3,raw_F5,raw_PO3,raw_PO4,raw_F6,...,gamma_7,SQ_CP3,SQ_C3,SQ_F5,SQ_PO3,SQ_PO4,SQ_F6,SQ_C4,SQ_CP4,frequency
0,raw,1.737637e+12,1.737637e+12,60100381,37.351214,34.872971,-10.157798,131.204983,16.797192,-245.138848,...,NaN,bad,bad,bad,bad,good,bad,bad,bad,10
1,raw,1.737637e+12,1.737637e+12,60100381,57.753821,27.546080,-6.372357,153.695031,19.685160,-245.883422,...,NaN,bad,bad,bad,bad,good,bad,bad,bad,10
2,raw,1.737637e+12,1.737637e+12,60100381,89.292359,19.793765,-3.722729,184.468183,21.563587,-248.808447,...,NaN,bad,bad,bad,bad,good,bad,bad,bad,10
3,raw,1.737637e+12,1.737637e+12,60100381,118.013263,8.508133,-6.230808,212.892665,20.493320,-258.419832,...,NaN,bad,bad,bad,bad,good,bad,bad,bad,10
4,raw,1.737637e+12,1.737637e+12,60100381,134.618681,-5.233543,-13.098480,231.765308,17.091776,-272.928715,...,NaN,bad,bad,bad,bad,good,bad,bad,bad,10


# Fixing Timestamp

In [6]:
df['timestamp_delta'] = df['timestamp'] - df['laptop_timestamp']

In [7]:
df['timestamp_delta'].describe()

# on average, the timestamp is 0.6 seconds ahead of the laptop timestamp

count    51155.000000
mean        14.517919
std        513.923814
min      -3206.000000
25%        103.509033
50%        185.735107
75%        225.819092
max        297.710938
Name: timestamp_delta, dtype: float64

In [8]:
df['timestamp'] = df['timestamp'] - 0.6
df.drop(columns=['timestamp_delta', 'laptop_timestamp'], inplace=True)

# Making Signal quality consolidated

In [9]:
def map_quality(value):
    if value <= 1.5:
        return 'bad'
    elif value <= 2.5:
        return 'good'
    else:
        return 'great'

cols = ['SQ_CP3', 'SQ_C3', 'SQ_F5', 'SQ_PO3', 'SQ_PO4', 'SQ_F6', 'SQ_C4', 'SQ_CP4']

quality_mapping = {'bad': 1, 'good': 2, 'great': 3}

for col in cols:
    df[col] = df[col].map(quality_mapping)
    
df['Average_Quality_Value'] = df[['SQ_CP3', 'SQ_C3', 'SQ_F5', 'SQ_PO3', 'SQ_PO4', 'SQ_F6', 'SQ_C4', 'SQ_CP4']].mean(axis=1)

df['Signal_Quality'] = df['Average_Quality_Value'].apply(map_quality)
df = df.drop(columns=['Average_Quality_Value', 'SQ_CP3', 'SQ_C3', 'SQ_F5', 'SQ_PO3', 'SQ_PO4', 'SQ_F6', 'SQ_C4', 'SQ_CP4'])

# Separating PowerByBand readings

In [18]:
df['type'].unique()

array(['raw', 'powerByBand'], dtype=object)

In [43]:
pbd_df = df[df['type'] == 'powerByBand'].dropna(axis=1).drop(columns=['type'])

In [44]:
cols = ['participantName', 'Signal_Quality'] + [col for col in pbd_df.columns if col not in ['participantName', 'Signal_Quality']]
pbd_df = pbd_df[cols]
pbd_df

,participantName,Signal_Quality,timestamp,alpha_0,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,alpha_6,...,beta_7,gamma_0,gamma_1,gamma_2,gamma_3,gamma_4,gamma_5,gamma_6,gamma_7,frequency
48,60100381,bad,1.737637e+12,11.523836,8.444360,2.468238,17.746076,2.060579,19.516475,12.329837,...,1.319532,1.634543,0.595011,0.315648,1.880328,0.391779,2.950650,1.524315,0.445237,10
113,60100381,bad,1.737637e+12,9.086326,1.354322,0.829728,9.270763,1.405224,8.838968,6.523937,...,1.460217,1.851547,0.512123,0.373625,1.940466,0.386018,1.111444,0.840030,0.389414,10
178,60100381,good,1.737637e+12,6.307510,2.219599,1.553624,9.090058,1.012324,0.760292,0.625772,...,0.832245,0.760794,0.377659,0.399477,1.249890,0.231642,0.412469,0.316035,0.333163,10
243,60100381,good,1.737637e+12,5.351354,2.191468,1.875632,7.465189,0.718275,1.555076,1.125960,...,0.909471,0.916484,0.386113,0.424669,1.567390,0.235702,0.357483,0.273637,0.424800,10
308,60100381,good,1.737637e+12,5.028265,1.386333,0.919287,4.695544,0.557102,2.219858,1.609974,...,0.840127,0.521087,0.284664,0.309306,0.968837,0.164732,0.354212,0.253030,0.178336,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50878,60100381,good,1.737638e+12,4.640241,0.942268,0.897851,3.986413,0.988467,0.976445,0.789891,...,0.907913,0.971806,0.274423,0.327329,1.059363,0.161075,0.254202,0.150222,0.262234,20
50943,60100381,good,1.737638e+12,3.126871,1.027702,1.093385,4.049445,0.410310,0.561761,0.880726,...,0.947441,1.148201,0.294591,0.369669,1.060959,0.167196,0.293482,0.247273,0.267661,20
51008,60100381,good,1.737638e+12,6.277153,0.887548,1.113298,4.245691,1.174204,0.776057,1.449232,...,1.154258,1.278990,0.341851,0.268741,1.296669,0.204760,0.255233,0.190736,0.349197,20
51073,60100381,good,1.737638e+12,3.649051,0.982815,0.425774,3.038952,0.682812,1.158992,0.838405,...,0.976744,1.284256,0.322485,0.265099,1.259195,0.222077,0.226340,0.163072,0.305754,20


In [45]:
pbd_df['timestamp_delta'] = pbd_df['timestamp'] - pbd_df['timestamp'].shift(1)
pbd_df['timestamp_delta'].describe()
# on average, 4 readings per second

count      786.000000
mean       314.250636
std       1054.632451
min        187.000000
25%        249.000000
50%        250.000000
75%        251.000000
max      21252.000000
Name: timestamp_delta, dtype: float64

In [46]:
pbd_df.drop(columns=['timestamp_delta'], inplace=True)

In [47]:
pbd_df.shape

(787, 28)

In [68]:
session_id = 1
sessions = []

for i in pbd_df['frequency'].unique():
    # Filter rows where frequency is 10
    filtered_df = pbd_df[pbd_df['frequency'] == i]

    # Create a new dataframe with sessions
    for i in range(0, len(filtered_df) - 3):
        session = filtered_df.iloc[i:i+4]
        if len(session) == 4:
            session['session_id'] = session_id
            sessions.append(session)
            session_id += 1

# Concatenate all sessions into a single dataframe
sessions_df = pd.concat(sessions)

# Reset index for the new dataframe and set session_id as the index
sessions_df.reset_index(drop=True, inplace=True)
sessions_df.set_index('session_id', inplace=True)

In [69]:
sessions_df

,participantName,Signal_Quality,timestamp,alpha_0,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,alpha_6,...,beta_7,gamma_0,gamma_1,gamma_2,gamma_3,gamma_4,gamma_5,gamma_6,gamma_7,frequency
session_id,,,,,,,,,,,,,,,,,,,,,
1,60100381,bad,1.737637e+12,11.523836,8.444360,2.468238,17.746076,2.060579,19.516475,12.329837,...,1.319532,1.634543,0.595011,0.315648,1.880328,0.391779,2.950650,1.524315,0.445237,10
1,60100381,bad,1.737637e+12,9.086326,1.354322,0.829728,9.270763,1.405224,8.838968,6.523937,...,1.460217,1.851547,0.512123,0.373625,1.940466,0.386018,1.111444,0.840030,0.389414,10
1,60100381,good,1.737637e+12,6.307510,2.219599,1.553624,9.090058,1.012324,0.760292,0.625772,...,0.832245,0.760794,0.377659,0.399477,1.249890,0.231642,0.412469,0.316035,0.333163,10
1,60100381,good,1.737637e+12,5.351354,2.191468,1.875632,7.465189,0.718275,1.555076,1.125960,...,0.909471,0.916484,0.386113,0.424669,1.567390,0.235702,0.357483,0.273637,0.424800,10
2,60100381,bad,1.737637e+12,9.086326,1.354322,0.829728,9.270763,1.405224,8.838968,6.523937,...,1.460217,1.851547,0.512123,0.373625,1.940466,0.386018,1.111444,0.840030,0.389414,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,60100381,good,1.737638e+12,3.649051,0.982815,0.425774,3.038952,0.682812,1.158992,0.838405,...,0.976744,1.284256,0.322485,0.265099,1.259195,0.222077,0.226340,0.163072,0.305754,20
775,60100381,good,1.737638e+12,3.126871,1.027702,1.093385,4.049445,0.410310,0.561761,0.880726,...,0.947441,1.148201,0.294591,0.369669,1.060959,0.167196,0.293482,0.247273,0.267661,20
775,60100381,good,1.737638e+12,6.277153,0.887548,1.113298,4.245691,1.174204,0.776057,1.449232,...,1.154258,1.278990,0.341851,0.268741,1.296669,0.204760,0.255233,0.190736,0.349197,20


In [70]:
sessions_df.to_csv('power_by_band.csv')

# Separating raw readings

In [176]:
df['type'].unique()

array(['raw', 'powerByBand'], dtype=object)

In [74]:
raw_df = df[df['type'] == 'raw'].dropna(axis=1).drop(columns=['type'])

In [75]:
cols = ['participantName', 'Signal_Quality'] + [col for col in raw_df.columns if col not in ['participantName', 'Signal_Quality']]
raw_df = raw_df[cols]
raw_df.head()

,participantName,Signal_Quality,timestamp,raw_CP3,raw_C3,raw_F5,raw_PO3,raw_PO4,raw_F6,raw_C4,raw_CP4,frequency
0,60100381,bad,1.737637e+12,37.351214,34.872971,-10.157798,131.204983,16.797192,-245.138848,-129.405742,-32.640629,10
1,60100381,bad,1.737637e+12,57.753821,27.546080,-6.372357,153.695031,19.685160,-245.883422,-138.472250,-32.341483,10
2,60100381,bad,1.737637e+12,89.292359,19.793765,-3.722729,184.468183,21.563587,-248.808447,-147.995575,-33.413265,10
3,60100381,bad,1.737637e+12,118.013263,8.508133,-6.230808,212.892665,20.493320,-258.419832,-160.770922,-39.569929,10
4,60100381,bad,1.737637e+12,134.618681,-5.233543,-13.098480,231.765308,17.091776,-272.928715,-175.844794,-50.188371,10


In [76]:
session_id = 1
sessions = []

for i in raw_df['frequency'].unique():
    filtered_df = raw_df[raw_df['frequency'] == i]

    # Create a new dataframe with sessions
    for i in range(0, len(filtered_df) - 255):
        session = filtered_df.iloc[i:i+256]
        if len(session) == 256:
            session['session_id'] = session_id
            sessions.append(session)
            session_id += 1

# Concatenate all sessions into a single dataframe
sessions_df = pd.concat(sessions)

# Reset index for the new dataframe and set session_id as the index
sessions_df.reset_index(drop=True, inplace=True)
sessions_df.set_index('session_id', inplace=True)

In [78]:
sessions_df

,participantName,Signal_Quality,timestamp,raw_CP3,raw_C3,raw_F5,raw_PO3,raw_PO4,raw_F6,raw_C4,raw_CP4,frequency
session_id,,,,,,,,,,,,
1,60100381,bad,1.737637e+12,37.351214,34.872971,-10.157798,131.204983,16.797192,-245.138848,-129.405742,-32.640629,10
1,60100381,bad,1.737637e+12,57.753821,27.546080,-6.372357,153.695031,19.685160,-245.883422,-138.472250,-32.341483,10
1,60100381,bad,1.737637e+12,89.292359,19.793765,-3.722729,184.468183,21.563587,-248.808447,-147.995575,-33.413265,10
1,60100381,bad,1.737637e+12,118.013263,8.508133,-6.230808,212.892665,20.493320,-258.419832,-160.770922,-39.569929,10
1,60100381,bad,1.737637e+12,134.618681,-5.233543,-13.098480,231.765308,17.091776,-272.928715,-175.844794,-50.188371,10
...,...,...,...,...,...,...,...,...,...,...,...,...
49348,60100381,good,1.737638e+12,24.655225,10.182872,7.427786,31.393160,9.204674,-0.743196,0.042608,7.313199,20
49348,60100381,good,1.737638e+12,28.148703,5.906303,5.088546,31.325913,6.152889,-3.955799,-3.063039,2.818513,20
49348,60100381,good,1.737638e+12,18.586736,1.038020,0.380562,18.477878,0.546905,-5.091733,-5.156195,-1.660931,20


In [79]:
sessions_df.to_csv('raw.csv')

In [ ]:
import pandas as pd
import warnings

def process_brainwaves_data(file_name):
    warnings.filterwarnings("ignore")

    # Read the data
    df = pd.read_csv(file_name)

    # Fixing Timestamp
    df['timestamp_delta'] = df['timestamp'] - df['laptop_timestamp']
    df['timestamp'] = df['timestamp'] - 0.6
    df.drop(columns=['timestamp_delta', 'laptop_timestamp'], inplace=True)

    # Making Signal quality consolidated
    def map_quality(value):
        if value <= 1.5:
            return 'bad'
        elif value <= 2.5:
            return 'good'
        else:
            return 'great'

    cols = ['SQ_CP3', 'SQ_C3', 'SQ_F5', 'SQ_PO3', 'SQ_PO4', 'SQ_F6', 'SQ_C4', 'SQ_CP4']
    quality_mapping = {'bad': 1, 'good': 2, 'great': 3}

    for col in cols:
        df[col] = df[col].map(quality_mapping)
    
    df['Average_Quality_Value'] = df[cols].mean(axis=1)
    df['Signal_Quality'] = df['Average_Quality_Value'].apply(map_quality)
    df = df.drop(columns=['Average_Quality_Value'] + cols)

    # Separating PowerByBand readings
    pbd_df = df[df['type'] == 'powerByBand'].dropna(axis=1).drop(columns=['type'])
    cols = ['participantName', 'Signal_Quality'] + [col for col in pbd_df.columns if col not in ['participantName', 'Signal_Quality']]
    pbd_df = pbd_df[cols]

    pbd_df['timestamp_delta'] = pbd_df['timestamp'] - pbd_df['timestamp'].shift(1)
    pbd_df.drop(columns=['timestamp_delta'], inplace=True)

    session_id = 1
    sessions = []

    for i in pbd_df['frequency'].unique():
        filtered_df = pbd_df[pbd_df['frequency'] == i]

        for i in range(0, len(filtered_df) - 3):
            session = filtered_df.iloc[i:i+4]
            if len(session) == 4:
                session['session_id'] = session_id
                sessions.append(session)
                session_id += 1

    sessions_df = pd.concat(sessions)
    sessions_df.reset_index(drop=True, inplace=True)
    sessions_df.set_index('session_id', inplace=True)
    sessions_df.to_csv('power_by_band.csv')

    # Separating raw readings
    raw_df = df[df['type'] == 'raw'].dropna(axis=1).drop(columns=['type'])
    cols = ['participantName', 'Signal_Quality'] + [col for col in raw_df.columns if col not in ['participantName', 'Signal_Quality']]
    raw_df = raw_df[cols]

    session_id = 1
    sessions = []

    for i in raw_df['frequency'].unique():
        filtered_df = raw_df[raw_df['frequency'] == i]

        for i in range(0, len(filtered_df) - 255):
            session = filtered_df.iloc[i:i+256]
            if len(session) == 256:
                session['session_id'] = session_id
                sessions.append(session)
                session_id += 1

    sessions_df = pd.concat(sessions)
    sessions_df.reset_index(drop=True, inplace=True)
    sessions_df.set_index('session_id', inplace=True)
    sessions_df.to_csv('raw.csv')

# Example usage
process_brainwaves_data("../data/uncleaned/brainwaves_data_60100381.csv")